## 记忆

Langgraph 的记忆分为短期记忆和长期记忆。

短期记忆是针对单个会话的，在会话中随时可以被调用。每次调用完图之后，会自动更新，然后在每个超步开始的时候读取。

长期记忆是跨会话线程的，可以在任何时间与任何线程中调用。

In [13]:
# 修复后的简化长期记忆管理

from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END, add_messages
from langchain_core.messages import HumanMessage, AIMessage
import json
import redis
from langchain_community.chat_models import ChatTongyi


# 状态定义
class GraphState(TypedDict):
    messages: Annotated[Sequence, add_messages]


# 大模型
llm = ChatTongyi(
    model_name="qwen-turbo",
    temperature=0.7,
    streaming=True
)


# 长期记忆管理类
class LongTermMemory:
    def __init__(self, session_id: str, url: str = "redis://localhost:6379/0"):
        self.session_id = session_id
        self.key = f"memory:{session_id}"
        try:
            self.redis_client = redis.from_url(url)
            self.redis_client.ping()
        except Exception as e:
            print(f"Redis连接失败: {e}")
            self.redis_client = None

    def save_memory(self, memory_type: str, content: str):
        """保存记忆"""
        if not self.redis_client:
            return
        try:
            memory_data = {
                "type": memory_type,
                "content": content
            }
            self.redis_client.lpush(self.key, json.dumps(memory_data))
            print(f"记忆已保存: {content[:30]}...")
        except Exception as e:
            print(f"保存记忆失败: {e}")

    def get_memory(self, limit: int = 10) -> list:
        """通过ID获取记忆"""
        if not self.redis_client:
            return []
        try:
            items = self.redis_client.lrange(self.key, 0, limit-1)
            memories = []
            for item in reversed(items):
                data = json.loads(item)
                if data["type"] == "human":
                    memories.append(HumanMessage(content=data["content"]))
                elif data["type"] == "ai":
                    memories.append(AIMessage(content=data["content"]))
            return memories
        except Exception as e:
            print(f"获取记忆失败: {e}")
            return []

    def clear_memory(self):
        """清除记忆"""
        if not self.redis_client:
            return
        try:
            self.redis_client.delete(self.key)
            print(f"已清除会话 {self.session_id} 的记忆")
        except Exception as e:
            print(f"清除记忆失败: {e}")


# 全局变量存储当前thread_id
current_thread_id = None

def chat_node(state: GraphState):
    global current_thread_id
    thread_id = current_thread_id or "default"
    
    # 初始化长期记忆
    memory = LongTermMemory(session_id=thread_id)
    
    # 获取当前用户消息
    user_message = state["messages"][-1]
    
    if isinstance(user_message, HumanMessage):
        # 保存用户消息到长期记忆
        memory.save_memory("human", user_message.content)
    
    # 获取历史记忆
    historical_messages = memory.get_memory(limit=10)
    
    # 构建对话上下文
    if historical_messages:
        all_messages = historical_messages + [user_message]
    else:
        all_messages = [user_message]
    
    print(f"使用 {len(all_messages)} 条消息作为上下文")
    
    # 调用模型生成回复
    response = llm.invoke(all_messages)
    print(f"AI回复: {response.content}")
    
    # 保存AI回复到长期记忆
    memory.save_memory("ai", response.content)
    
    return {"messages": [response]}


# 构建图
builder = StateGraph(GraphState)
builder.add_node("chat", chat_node)
builder.add_edge(START, "chat")
builder.add_edge("chat", END)

from langgraph.checkpoint.memory import MemorySaver
app = builder.compile(checkpointer=MemorySaver())


# 辅助函数
def show_memory(thread_id: str):
    """显示指定ID的记忆"""
    memory = LongTermMemory(session_id=thread_id)
    messages = memory.get_memory(limit=20)
    
    print(f"\n=== 会话 '{thread_id}' 的记忆 ===")
    if not messages:
        print("无记录")
    else:
        for i, msg in enumerate(messages, 1):
            role = "用户" if isinstance(msg, HumanMessage) else "助手"
            print(f"{role} {i}: {msg.content}")


def delete_memory(thread_id: str):
    """删除指定ID的记忆"""
    memory = LongTermMemory(session_id=thread_id)
    memory.clear_memory()


# 测试长期记忆
def test_long_term_memory():
    global current_thread_id
    
    print("开始测试长期记忆系统")
    
    # Alice 的对话
    print("\nAlice 第一次对话:")
    current_thread_id = "alice"
    app.invoke(
        {"messages": [HumanMessage(content="我是Alice，我喜欢读书")]},
        config={"configurable": {"thread_id": "alice"}}
    )
    
    print("Alice 第二次对话:")
    current_thread_id = "alice"
    app.invoke(
        {"messages": [HumanMessage(content="我刚才说喜欢什么？")]},
        config={"configurable": {"thread_id": "alice"}}
    )
    
    # Bob 的对话
    print("\nBob 第一次对话:")
    current_thread_id = "bob"
    app.invoke(
        {"messages": [HumanMessage(content="我是Bob，我喜欢运动")]},
        config={"configurable": {"thread_id": "bob"}}
    )
    
    print("Bob 第二次对话:")
    current_thread_id = "bob"
    app.invoke(
        {"messages": [HumanMessage(content="我叫什么名字？")]},
        config={"configurable": {"thread_id": "bob"}}
    )
    
    # 显示记忆
    show_memory("alice")
    show_memory("bob")
    
    # 清除Alice的记忆
    print("\n清除Alice的记忆")
    delete_memory("alice")
    show_memory("alice")


if __name__ == "__main__":
    test_long_term_memory()


开始测试长期记忆系统

Alice 第一次对话:
记忆已保存: 我是Alice，我喜欢读书...
使用 2 条消息作为上下文
AI回复: 你好，Alice！很高兴认识你。你喜欢读书，那一定读过很多有趣的书吧？我特别想知道你最近在读什么书，或者有没有特别喜欢的类型？比如小说、科幻、历史还是其他类型的书呢？
记忆已保存: 你好，Alice！很高兴认识你。你喜欢读书，那一定读过很多有...
Alice 第二次对话:
记忆已保存: 我刚才说喜欢什么？...
使用 4 条消息作为上下文
AI回复: 你刚才说：“我是Alice，我喜欢读书。” 所以你刚才说的是你喜欢读书。😊
记忆已保存: 你刚才说：“我是Alice，我喜欢读书。” 所以你刚才说的是...

Bob 第一次对话:
记忆已保存: 我是Bob，我喜欢运动...
使用 2 条消息作为上下文
AI回复: 你好，Bob！很高兴认识你。喜欢运动是个非常好的习惯，能让人保持健康和活力。你平时都喜欢哪些运动呢？是喜欢跑步、打球，还是健身、游泳之类的？我很好奇你的运动经历和爱好！😊
记忆已保存: 你好，Bob！很高兴认识你。喜欢运动是个非常好的习惯，能让人...
Bob 第二次对话:
记忆已保存: 我叫什么名字？...
使用 4 条消息作为上下文
AI回复: 你叫Bob呀！之前你已经告诉过我了，说"我是Bob，我喜欢运动"。所以，我当然知道你的名字是Bob啦！😊 你是不是在测试我的记忆力呢？不过没关系，我很乐意和你聊天，无论你问什么问题，我都会认真回答的！
记忆已保存: 你叫Bob呀！之前你已经告诉过我了，说"我是Bob，我喜欢运...

=== 会话 'alice' 的记忆 ===
用户 1: 我是Alice，我喜欢读书
助手 2: 你好，Alice！很高兴认识你。你喜欢读书，那一定读过很多有趣的书吧？我特别想知道你最近在读什么书，或者有没有特别喜欢的类型？比如小说、科幻、历史还是其他类型的书呢？
用户 3: 我刚才说喜欢什么？
助手 4: 你刚才说：“我是Alice，我喜欢读书。” 所以你刚才说的是你喜欢读书。😊

=== 会话 'bob' 的记忆 ===
用户 1: 我是Bob，我喜欢运动
助手 2: 你好，Bob！很高兴认识你。喜欢运动是个非常好的习惯，能让人保持健康和活力。你平时都喜欢哪些运动呢？是喜欢跑步、打球，还是健